In [2]:
import pandas as pd

In [2]:
pd.__version__

'1.4.2'

In [3]:
!pip install pyarrow

In [3]:
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [11]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [12]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.40,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.00,-1.00,-0.5,0.00,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.00,1.00,0.5,0.00,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.90,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.00,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2,2023-02-28 23:46:00,2023-03-01 00:05:00,NaN,4.65,NaN,None,249,140,0,20.22,0.00,0.5,4.84,0.0,1.0,29.06,NaN,NaN
2913951,2,2023-02-28 23:26:02,2023-02-28 23:37:10,NaN,2.47,NaN,None,186,79,0,13.66,0.00,0.5,2.65,0.0,1.0,20.31,NaN,NaN
2913952,2,2023-02-28 23:24:00,2023-02-28 23:38:00,NaN,3.49,NaN,None,158,143,0,17.64,0.00,0.5,0.00,0.0,1.0,21.64,NaN,NaN
2913953,2,2023-02-28 23:03:00,2023-02-28 23:10:00,NaN,2.13,NaN,None,79,162,0,13.56,0.00,0.5,2.63,0.0,1.0,20.19,NaN,NaN


In [13]:
# Calculate the number of columns
num_columns = df.shape[1]

# Print the number of columns
print(f"The number of columns in the DataFrame is: {num_columns}")

The number of columns in the DataFrame is: 19


In [4]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [5]:
# Convert the pickup and dropoff times to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Compute the duration of each trip in minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Calculate the standard deviation of the trip durations
duration_std_dev = df['duration'].std()

print(f"The standard deviation of the trip durations is: {duration_std_dev} minutes")

The standard deviation of the trip durations is: 42.594351241920904 minutes


In [6]:
# Filter the DataFrame to include only records where the duration is between 1 and 60 minutes
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Calculate the fraction of records left after dropping outliers
fraction_left = len(filtered_df) / len(df)

print(f"Fraction of records left after dropping outliers: {fraction_left:.2%}")

Fraction of records left after dropping outliers: 98.12%


In [7]:
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

# Cast the pickup and dropoff location IDs to strings using .loc
filtered_df.loc[:, 'PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df.loc[:, 'DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# Create a list of dictionaries
data_dict = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer(sparse=True)
feature_matrix = dv.fit_transform(data_dict)

# Get the dimensionality of the feature matrix
num_columns = feature_matrix.shape[1]

num_columns

515

In [10]:
X = dv.fit_transform(data_dict)

# Response variable
y = filtered_df['duration']

# Train a linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict on the training data
y_pred = model.predict(X)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))

rmse

7.647460801490655

In [8]:
# Reduce the size of the dataset to avoid memory issues

filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

filtered_df = filtered_df.sample(frac=0.1, random_state=42)

# Cast the pickup and dropoff location IDs to strings using .loc
filtered_df.loc[:, 'PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df.loc[:, 'DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# Create a list of dictionaries
data_dict = filtered_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer(sparse=True)

X = dv.fit_transform(data_dict)

# Response variable
y = filtered_df['duration']

# Train a linear regression model
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [9]:
val_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"
df_val = pd.read_parquet(val_url)

# Preprocess the validation data
df_val['tpep_pickup_datetime'] = pd.to_datetime(df_val['tpep_pickup_datetime'])
df_val['tpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'])
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60
filtered_df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)].copy()

# Reduce the size of the dataset to avoid memory issues
filtered_df_val = filtered_df_val.sample(frac=0.1, random_state=42)

filtered_df_val.loc[:, 'PULocationID'] = filtered_df_val['PULocationID'].astype(str)
filtered_df_val.loc[:, 'DOLocationID'] = filtered_df_val['DOLocationID'].astype(str)

# Create a list of dictionaries for the validation data
val_data_dict = filtered_df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Transform the validation data using the fitted dictionary vectorizer
X_val = dv.transform(val_data_dict)

# Response variable for the validation data
y_val = filtered_df_val['duration']

# Predict on the validation data
y_val_pred = model.predict(X_val)

# Calculate the RMSE on the validation data
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))

print(f"RMSE on validation data: {rmse_val}")

RMSE on validation data: 7.8262503731342905
